### Tuning Hyper parameter

In [1]:
import tensorflow as tf
from tensorflow import keras
from model_bulid import build_model
from kerastuner.tuners import BayesianOptimization
from kerastuner import HyperParameters
import sklearn 
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

# One-hot encode the target variable
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

# validation dataset

X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.3)

# Define the search space
hp = HyperParameters()
hp.Choice('activation_fn', ['tanh', 'relu'])
hp.Int('hidden_node', 5, 11, 1)
hp.Choice('init_weight', ['random_normal', 'glorot_uniform', 'he_uniform'])
hp.Choice('optimizer_type', ['SGD', 'Adam'])
hp.Float('regularization_coefficient', 0.0001, 0.001,step=0.0009)
hp.Float('learning_rate', 0.0001, 0.1,step=0.099)
hp.Choice('learning_decay_schedule', ['none', 'cosine'])
hp.Choice('loss_function', ['categorical_crossentropy', 'mean_squared_error'])

# Define the tuner
tuner = BayesianOptimization(
    build_model,
    hyperparameters=hp,
    objective='val_accuracy',
    max_trials=3,
    num_initial_points=2,
    overwrite=True,
    directory='bayesian_opt',
    project_name='mnist'
)

# Define a function to print the details of each trial
def on_trial_end(trial):
    print(f'Trial {trial.trial_id}:\n')
    print(f'Hyperparameters: {trial.hyperparameters}\n')
    print(f'Metrics: {trial.metrics}\n')

# Search for the best hyperparameters
tuner.search(
    x=X_train,
    y=y_train,
    epochs=hp.Int('epochs',100,300,step=100),
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[keras.callbacks.LambdaCallback(on_trial_end=on_trial_end)]
)

# Print the best hyperparameters
print(tuner.get_best_hyperparameters()[0].values)

Trial 3 Complete [00h 06m 59s]
val_accuracy: 0.920722246170044

Best val_accuracy So Far: 0.920722246170044
Total elapsed time: 00h 22m 56s
INFO:tensorflow:Oracle triggered exit
{'activation_fn': 'relu', 'hidden_node': 8, 'init_weight': 'random_normal', 'optimizer_type': 'SGD', 'regularization_coefficient': 0.0001, 'learning_rate': 0.09910000000000001, 'learning_decay_schedule': 'cosine', 'loss_function': 'categorical_crossentropy', 'regularization_coefficients': 0.001, 'decay_schedule': 'none', 'epochs': 200}


### see performence

In [5]:
top_models = tuner.get_best_models(num_models=3)
top_parameters = tuner.get_best_hyperparameters(num_trials=3)
models = list(map(lambda x: x ,top_models))
top = models[0]
second = models[1]
third = models[2]

# Evaluate the model on the test set
loss_top, accuracy_top =top.evaluate(X_test, y_test)
loss_second, accuracy_second =second.evaluate(X_test, y_test)
loss_third, accuracy_third =third.evaluate(X_test, y_test)
print(f'top model Test accuracy:{accuracy_top}')
print(f'top model loss : {loss_top}')
print('\n')
print(f'second model Test accuracy:{accuracy_second}')
print(f'second model loss : {loss_second}')
print('\n')
print(f'third model Test accuracy:{accuracy_third}')
print(f'third model loss : {loss_third}')
print('\n')
ensemble_mean = (accuracy_top + accuracy_second + accuracy_third)/3
print(f'ensemble_mean：{ensemble_mean}')

313/313 [==============================] - 1s 2ms/step - loss: 0.5452 - accuracy: 0.8645
top model Test accuracy:0.9240000247955322
top model loss : 0.31067758798599243


second model Test accuracy:0.8952000141143799
second model loss : 0.3842472732067108


third model Test accuracy:0.8644999861717224
third model loss : 0.5452260375022888


ensemble_mean：0.8945666750272115


### Save model (weight&model)

In [6]:
top.save('2_layer_top_NNmodel.tf')
top.save_weights('2_layer_top_NNmodel_weight.tf')
second.save('2_layer_second_NNmodel.tf')
second.save_weights('2_layer_second_NNmodel_weight.tf')
third.save('2_layer_third_NNmodel.tf')
third.save_weights('2_layer_third_NNmodel_weight.tf')


INFO:tensorflow:Assets written to: 2_layer_top_NNmodel.tf\assets
INFO:tensorflow:Assets written to: 2_layer_second_NNmodel.tf\assets
INFO:tensorflow:Assets written to: 2_layer_third_NNmodel.tf\assets


### End

In [4]:
import tensorflow as tf

# Get all available devices
devices = tf.config.list_physical_devices()

# Filter only GPUs
gpus = [device for device in devices if device.device_type == 'GPU']

# Print the list of GPUs
print("Available GPUs:", gpus)


Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


In [6]:
with tf.device('/gpu:0'):
    model = create_model()
    train_dataset = create_train_dataset()


NameError: name 'create_model' is not defined

In [7]:
import tensorflow as tf

# Check available physical devices
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Get GPU device name
if len(tf.config.list_physical_devices('GPU')) > 0:
    gpu_name = tf.test.gpu_device_name()
    if gpu_name != '':
        print('Found GPU at: {}'.format(gpu_name))
    else:
        print("GPU device not found.")


Num GPUs Available:  1
Found GPU at: /device:GPU:0


In [8]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print(f"Number of GPUs available: {len(physical_devices)}")
for device in physical_devices:
    print(tf.config.experimental.get_device_details(device))


Number of GPUs available: 1
{'device_name': 'NVIDIA GeForce RTX 3060 Laptop GPU', 'compute_capability': (8, 6)}
